The code provided is for setting up PySpark in a Google Colab environment or a similar setup where PySpark is not pre-installed.

**!apt-get install openjdk-8-jdk-headless -qq > /dev/null**
Uses the apt-get package manager to install OpenJDK 8 without the graphical user interface components (headless).
The -qq flag minimizes the output, and > /dev/null suppresses it entirely.

**!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz**
Uses wget to download the specified version of Apache Spark (3.1.1) with Hadoop 3.2 support from the Apache archive.
The -q flag makes the download process quiet (minimal output).

**!tar xf spark-3.1.1-bin-hadoop3.2.tgz**
Uses the tar command to extract the downloaded tarball file (spark-3.1.1-bin-hadoop3.2.tgz).

**!pip install -q findspark**
Installs the findspark library using pip to facilitate the integration of Spark with Python

**import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"**
Sets the JAVA_HOME environment variable to the path where OpenJDK 8 is installed.
Sets the SPARK_HOME environment variable to the path where Spark was extracted.

**import findspark
findspark.init()**
Initializes findspark to make PySpark available in the Python environment.

**from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark
Imports SparkSession from pyspark.sql.**

Builds and initializes a Spark session with the master URL set to local[*], which means Spark will run locally with as many worker threads as logical cores on the machine.
The getOrCreate() method either retrieves an existing Spark session or creates a new one if none exist.

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark


In [ ]:
mydata = spark.read.format("csv").option("header", "true").load("/content/original.csv")
mydata.show()

# spark.read.format("csv"): Specifies that the input file is in CSV format.
# option("header", "true"): Indicates that the CSV file contains a header row.
# load("/content/original.csv"): Loads the CSV file from the specified path into a DataFrame.

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [ ]:
from pyspark.sql.functions import *

mydata2 = mydata.withColumn("clean_city",when(mydata.City.isNull(),"Unknown").otherwise(mydata.City))
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|        Unknown|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|
|  6|     Maris|      Folk|Femal

In [ ]:
# mydata2 = mydata.withColumn("clean_JobTitle",when(mydata.JobTitle.isNull(),"Unknown").otherwise(mydata.JobTitle))
# mydata2.show()

In [ ]:
mydata2 = mydata2.filter(mydata2.JobTitle.isNotNull()) #FILTER DATA WHERE JOBTITLE IS NOT NULL
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|Kinsealy-Drinan|
|  8|   Goddart|     Flear|  Mal

In [ ]:
# CLEAN THE SALARY COLUMN TO REMOVE THE $ SYMBOL

mydata2 = mydata2.withColumn("clean_salary",regexp_replace(mydata2.Salary, '[$,]', ''))
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|    62846.60|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|    63863.09|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil 

In [ ]:
mydata2 = mydata2.withColumn("clean_salary_2",mydata2.Salary.substr(2,100).cast("float"))
mydata2.show()


+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+--------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|clean_salary_2|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+--------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|      57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|    62846.60|       62846.6|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|      61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|  

In [ ]:
mean =mydata2.groupBy().avg("clean_salary_2")
mean.show()

+-------------------+
|avg(clean_salary_2)|
+-------------------+
|  55516.32088199837|
+-------------------+



In [ ]:
meanval = mydata2.groupBy().avg("clean_salary_2").take(1)[0][0] #take the first row ,1st column value
meanval
# It appears that the variable mean is being overwritten in the last cell of code.
# It's initially a DataFrame, but then it's reassigned to a float value representing the average salary.
# The show() method is applicable to DataFrames, but not to float values

55516.32088199837

In [ ]:
mean_df = mydata2.groupBy().avg("clean_salary_2").take(1)[0]
mean = mean_df[0] # Extract the float value

print(mean) # Print the mean value

55516.32088199837


In [ ]:
from pyspark.sql.functions import lit #Creates a ~pyspark.sql.Column of literal value.

mydata2 = mydata2.withColumn("new_salary",when(mydata2.clean_salary.isNull(),lit(mean)).otherwise(mydata2.clean_salary))
#create a new column with salary, if salary is null, replace i with mean value calculated else the same column value

mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+--------------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|clean_salary_2|new_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+--------------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|      57438.18|  57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|    62846.60|       62846.6|  62846.60|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|      61489.23|  61489.23|
|  5|  Sherwood|   Macieja|  Male|      

In [ ]:
import numpy as np
latitudes = mydata2.select("Latitude")
longitudes = mydata2.select("Longitude")
latitudes.show()
longitudes.show()

+----------+
|  Latitude|
+----------+
|50.5774075|
|48.8231572|
|44.5047212|
|      null|
|53.4266145|
|45.1905186|
| 32.027934|
|  4.272793|
|     -5.85|
| 39.172378|
|49.8151822|
|42.1014803|
|49.7923299|
|43.4945737|
|52.7441662|
| 38.696249|
|-7.7232567|
|40.7172049|
|  49.16291|
|40.7576842|
+----------+
only showing top 20 rows

+-----------+
|  Longitude|
+-----------+
| 16.4967184|
|103.5218199|
| 38.1300171|
| 37.6489954|
| -6.1644997|
|  0.7423124|
| 106.657113|
| -74.416014|
|-79.8833329|
| 116.931607|
| 19.3771749|
|-72.5766759|
| 13.4915324|
|  5.8978018|
| 17.3278637|
| -8.7098337|
|113.4686802|
| -8.3625148|
|  127.98658|
| 23.1342184|
+-----------+
only showing top 20 rows



In [ ]:
latitudes = latitudes.filter(latitudes.Latitude.isNotNull())
longitudes = longitudes.filter(longitudes.Longitude.isNotNull())
latitudes.show()
longitudes.show()

+----------+
|  Latitude|
+----------+
|50.5774075|
|48.8231572|
|44.5047212|
|53.4266145|
|45.1905186|
| 32.027934|
|  4.272793|
|     -5.85|
| 39.172378|
|49.8151822|
|42.1014803|
|49.7923299|
|43.4945737|
|52.7441662|
| 38.696249|
|-7.7232567|
|40.7172049|
|  49.16291|
|40.7576842|
|48.4902808|
+----------+
only showing top 20 rows

+-----------+
|  Longitude|
+-----------+
| 16.4967184|
|103.5218199|
| 38.1300171|
| 37.6489954|
| -6.1644997|
|  0.7423124|
| 106.657113|
| -74.416014|
|-79.8833329|
| 116.931607|
| 19.3771749|
|-72.5766759|
| 13.4915324|
|  5.8978018|
| 17.3278637|
| -8.7098337|
|113.4686802|
| -8.3625148|
|  127.98658|
| 23.1342184|
+-----------+
only showing top 20 rows



In [ ]:
latitudes = latitudes.withColumn("latitude1",latitudes.Latitude.cast("float")).select("latitude1")
longitudes = longitudes.withColumn("longitude1",longitudes.Longitude.cast("float")).select("longitude1")
latitudes.show()

+----------+
| latitude1|
+----------+
| 50.577408|
|  48.82316|
| 44.504723|
| 53.426613|
| 45.190517|
| 32.027935|
|  4.272793|
|     -5.85|
|  39.17238|
|  49.81518|
|  42.10148|
|  49.79233|
| 43.494576|
| 52.744167|
| 38.696247|
|-7.7232566|
| 40.717205|
|  49.16291|
| 40.757683|
|  48.49028|
+----------+
only showing top 20 rows



In [ ]:
median = np.median(latitudes.collect())
#  the collect() method is used to retrieve the entire dataset (or the elements of an RDD or DataFrame) from the Spark cluster to the local machine as a list.
print(median)

31.93397331237793


In [ ]:
mydata2 = mydata2.withColumn("new_latitude",when(mydata2.Latitude.isNull(),lit(median)).otherwise(mydata2.Latitude))
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+--------------+----------+-----------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|clean_salary_2|new_salary|     new_latitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+--------------+----------+-----------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|      57438.18|  57438.18|       50.5774075|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|    62846.60|       62846.6|  62846.60|       48.8231572|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divn

# READ DATA USING SPARK AND CLEANING IT IN DIFFERENT & EASY WAY

In [ ]:
df2 = spark.read.csv("/content/original.csv", header=True)
df2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [ ]:
df2.dtypes

In [ ]:
from pyspark.sql.types import *
myschema = StructType([
    StructField("id", IntegerType(), True),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("City", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("JobTitle", StringType(), True),
    StructField("Salary", StringType(), True),
    StructField("Latitude", StringType(), True),
    StructField("Longitude", FloatType(), True)

])

In [ ]:
df3 = spark.read.csv("/content/original.csv", header=True, schema=myschema)
df3.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|  City|         gender|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|              

In [ ]:
df3.dtypes

[('id', 'int'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('City', 'string'),
 ('gender', 'string'),
 ('JobTitle', 'string'),
 ('Salary', 'string'),
 ('Latitude', 'string'),
 ('Longitude', 'float')]

## OVERALL MEN OR WOMEN GET PAID MORE ON AVG
## BY JOB TITLE MEN OR WOMEN GET PAID MORE ON AVG
## WHICH CITY HAS THE HIGHEST AVERAGE SALARY


In [ ]:
import pyspark.sql.functions as sqlfunc
genders = mydata2.groupBy("gender").agg(sqlfunc.avg("new_salary")).alias("AvgSalary")
genders.show()

+------+-----------------+
|gender|  avg(new_salary)|
+------+-----------------+
|Female|55677.25018367348|
|  Male|55361.09383858273|
+------+-----------------+



In [ ]:
jobtitleAvgSal = mydata2.groupBy("JobTitle").agg(sqlfunc.avg("new_salary")).alias("AvgSalary")
genders.show()


+------+-----------------+
|gender|  avg(new_salary)|
+------+-----------------+
|Female|55677.25018367348|
|  Male|55361.09383858273|
+------+-----------------+



In [ ]:
df4= mydata2.withColumn("Fem_salary",when(mydata2.gender=="Female",mydata2.new_salary).otherwise(lit(0)))
df4.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+--------------+----------+-----------------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|clean_salary_2|new_salary|     new_latitude|Fem_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+--------------+----------+-----------------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|      57438.18|  57438.18|       50.5774075|  57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|    62846.60|       62846.6|  62846.60|       48.8231572|  62846.60|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budge

In [ ]:
df4= df4.withColumn("male_salary",when(df4.gender=="Male",df4.new_salary).otherwise(lit(0)))
df4.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+--------------+----------+-----------------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|clean_salary_2|new_salary|     new_latitude|Fem_salary|male_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+--------------+----------+-----------------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|      57438.18|  57438.18|       50.5774075|  57438.18|          0|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|    62846.60|       62846.6|  62846.60|       48.8231572|  62846.60|        

In [ ]:
df5 = df4.groupBy("JobTitle").agg(sqlfunc.avg("Fem_salary").alias("FemaleAvgSalary") , sqlfunc.avg("Male_salary").alias("MaleAvgSalary"))
df5.show()

+--------------------+------------------+------------------+
|            JobTitle|   FemaleAvgSalary|     MaleAvgSalary|
+--------------------+------------------+------------------+
|Systems Administr...|         50590.475|          15540.95|
|   Media Manager III|29586.436666666665|17381.920000000002|
|  Recruiting Manager| 34848.45166666667|         26383.495|
|       Geologist III|31749.046666666665|12830.753333333332|
|        Geologist II|               0.0|         43293.865|
|Database Administ...|               0.0|          52018.46|
|   Financial Analyst|         23353.777|         39606.057|
|  Analyst Programmer|         16406.129|21042.962999999996|
|Software Engineer II|               0.0|          74782.64|
|       Accountant IV|          82732.25|               0.0|
|    Product Engineer|41825.484000000004|         20464.944|
|Software Test Eng...|32218.608333333337|27122.463333333333|
|Safety Technician...|               0.0|          29421.53|
|    Junior Executive|15

In [ ]:
df5 = df4.withColumn("diff Salary",df4.Fem_salary-df4.male_salary)
df5.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+--------------+----------+-----------------+----------+-----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|clean_salary_2|new_salary|     new_latitude|Fem_salary|male_salary|diff Salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+--------------+----------+-----------------+----------+-----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|      57438.18|  57438.18|       50.5774075|  57438.18|          0|   57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|    62846.60|       62846.6|

In [ ]:
CityHighSal = df5.groupBy("City").agg(sqlfunc.avg("new_salary").alias("AvgCitySalary")).sort("AvgCitySalary",ascending=False)
CityHighSal.show()

+-----------------+-------------+
|             City|AvgCitySalary|
+-----------------+-------------+
|        Mesopotam|     99948.28|
|       Zhongcheng|     99942.92|
|           Caxias|      99786.4|
|      Karangtawar|     99638.99|
|        Itabaiana|     99502.16|
|           Pasian|     99421.34|
|           Webuye|     99368.55|
|      Yuktae-dong|     99250.83|
|           Zinder|     99222.84|
|   Timiryazevskiy|     99142.94|
|        Sawahbaru|     99013.71|
|          Madimba|     98737.87|
|         Huangshi|     98690.34|
|          Gharyan|     98679.31|
|         Yŏnan-ŭp|     98628.61|
|     Wringinputih|     98603.82|
|Monte da Boavista|     98586.72|
|          Klukeng|     98439.49|
|         Murmashi|     98226.16|
|        Fox Creek|      98138.0|
+-----------------+-------------+
only showing top 20 rows



In [ ]:
# OR
CityHighSal = df5.groupBy("City").agg(sqlfunc.avg("new_salary").alias("AvgCitySalary"))
CityHighSal.sort(col("AvgCitySalary").desc())
CityHighSal.show()

+-----------------+-------------+
|             City|AvgCitySalary|
+-----------------+-------------+
|        Sułkowice|     33432.99|
|          Klippan|     77039.47|
|      Trollhättan|    53311.685|
|        Shinaihai|     39544.64|
|         Hongzhou|     35707.31|
|         Cipinang|     11617.51|
| Viejo Daan Banua|      43927.5|
|         Tsiatsan|     18795.44|
|       San Andres|      52426.8|
|           Krasna|     72022.79|
|      Springfield|    40697.325|
|            Město|     27797.98|
|Chaloem Phra Kiat|      54840.2|
|          Tadotsu|      55595.3|
|   Hénin-Beaumont|     55082.75|
|          Kajaani|     20224.84|
|           Duozhu|     71416.86|
|           Abéché|     93375.18|
|     Habingkloang|     56892.97|
|         Malishka|     76783.48|
+-----------------+-------------+
only showing top 20 rows



In [ ]:
df2.head()

Row(id='1', first_name='Melinde', last_name='Shilburne', gender='Female', City='Nowa Ruda', JobTitle='Assistant Professor', Salary='$57438.18', Latitude='50.5774075', Longitude='16.4967184')

In [ ]:
df2.head(10)

[Row(id='1', first_name='Melinde', last_name='Shilburne', gender='Female', City='Nowa Ruda', JobTitle='Assistant Professor', Salary='$57438.18', Latitude='50.5774075', Longitude='16.4967184'),
 Row(id='2', first_name='Kimberly', last_name='Von Welden', gender='Female', City='Bulgan', JobTitle='Programmer II', Salary='$62846.60', Latitude='48.8231572', Longitude='103.5218199'),
 Row(id='3', first_name='Alvera', last_name='Di Boldi', gender='Female', City=None, JobTitle=None, Salary='$57576.52', Latitude='39.9947462', Longitude='116.3397725'),
 Row(id='4', first_name='Shannon', last_name="O'Griffin", gender='Male', City='Divnomorskoye', JobTitle='Budget/Accounting Analyst II', Salary='$61489.23', Latitude='44.5047212', Longitude='38.1300171'),
 Row(id='5', first_name='Sherwood', last_name='Macieja', gender='Male', City='Mytishchi', JobTitle='VP Sales', Salary='$63863.09', Latitude=None, Longitude='37.6489954'),
 Row(id='6', first_name='Maris', last_name='Folk', gender='Female', City='Kin

In [ ]:
df2.describe()

DataFrame[summary: string, id: string, first_name: string, last_name: string, gender: string, City: string, JobTitle: string, Salary: string, Latitude: string, Longitude: string]

In [ ]:
df2.summary()

DataFrame[summary: string, id: string, first_name: string, last_name: string, gender: string, City: string, JobTitle: string, Salary: string, Latitude: string, Longitude: string]

In [ ]:
df2.first()

Row(id='1', first_name='Melinde', last_name='Shilburne', gender='Female', City='Nowa Ruda', JobTitle='Assistant Professor', Salary='$57438.18', Latitude='50.5774075', Longitude='16.4967184')

In [ ]:
df2.describe().show()

+-------+-----------------+----------+---------+------+-------------------+-------------------+---------+-----------------+------------------+
|summary|               id|first_name|last_name|gender|               City|           JobTitle|   Salary|         Latitude|         Longitude|
+-------+-----------------+----------+---------+------+-------------------+-------------------+---------+-----------------+------------------+
|  count|             1000|      1000|     1000|  1000|                999|                998|     1000|              999|              1000|
|   mean|            500.5|      null|     null|  null|               null|               null|     null|25.43151724234234|43.337564614499996|
| stddev|288.8194360957494|      null|     null|  null|               null|               null|     null| 24.5790825486909| 69.42064539970089|
|    min|                1|   Abagail|    Abbay|Female|             Abéché|Account Coordinator|$10101.92|       -0.6256517|        -0.4889547|

In [ ]:
df2.columns

['id',
 'first_name',
 'last_name',
 'gender',
 'City',
 'JobTitle',
 'Salary',
 'Latitude',
 'Longitude']

In [ ]:
df2.count() #number of rows

1000

In [ ]:
df2.distinct().count() #number of unique rows

1000

In [ ]:
df_dropped = df2.na.drop() #drop rows with missing values
df_dropped.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  8|   Goddart|     Flear|  Male|      Trélissac|Desktop Support T...|$46116.36|45.1905186|  0.7423124|
|  9|      Roth|O'Cannavan|  Male|         Heitan|VP Product Manage...|$73697.10| 32.027934| 106.657113|
| 10|      Bran|   Trahear|  Male|       Arbeláez|Mecha

In [ ]:
df_dropped.count()

997

In [ ]:
df_off_duplicates = df_dropped.dropDuplicates() #drop duplicates
df_off_duplicates.show()

+---+----------+----------+------+----------------+--------------------+---------+----------+------------+
| id|first_name| last_name|gender|            City|            JobTitle|   Salary|  Latitude|   Longitude|
+---+----------+----------+------+----------------+--------------------+---------+----------+------------+
|167|    Buiron| Franzotto|  Male|         Prusice|    Health Coach III|$55731.78|51.3704296|  16.9606267|
|610|    Waylen| O' Clovan|  Male|           Khyzy|   Marketing Manager|$73169.48|40.9109489|  49.0729264|
|239|  Vivyanne|  Astridge|Female|         Baoshui|        Food Chemist|$11084.34| 30.513598|  110.361715|
|256|    Stearn|    Niesel|  Male|         Lungmar| Software Consultant|$31406.56|  28.85816|    89.94036|
|575|    Richie|  Hellicar|  Male|         Xichuan|        Developer IV|$40279.59|  33.13782|  111.490964|
|578|    Mignon|    Debnam|Female|   Guajará Mirim|       Social Worker|$81607.74|-10.789321| -65.3301049|
|784|  Marjorie|  Dekeyser|Female|   

In [ ]:
df_off_duplicates.count()

997

In [ ]:
df_renamed = df2.withColumnRenamed("first_name","fn")
df_renamed.show()

+---+--------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|      fn| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+--------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1| Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|  Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4| Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|   Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|   Masha|    Divers|Female|         Dachun|                null|$250

In [ ]:
df_filter = df2.filter(df2.first_name=="Alvera")
df_filter.show()

+---+----------+---------+------+----+--------+---------+----------+-----------+
| id|first_name|last_name|gender|City|JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+---------+------+----+--------+---------+----------+-----------+
|  3|    Alvera| Di Boldi|Female|null|    null|$57576.52|39.9947462|116.3397725|
+---+----------+---------+------+----+--------+---------+----------+-----------+



In [ ]:
df_filter = df2.filter(df2.first_name.like("%vera"))
df_filter.show()

+---+----------+---------+------+----+--------+---------+----------+-----------+
| id|first_name|last_name|gender|City|JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+---------+------+----+--------+---------+----------+-----------+
|  3|    Alvera| Di Boldi|Female|null|    null|$57576.52|39.9947462|116.3397725|
+---+----------+---------+------+----+--------+---------+----------+-----------+



In [ ]:
df_filter = df2.filter(df2.first_name.endswith("non"))
df_filter.show()

+---+----------+---------+------+-------------+--------------------+---------+----------+-----------+
| id|first_name|last_name|gender|         City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+---------+------+-------------+--------------------+---------+----------+-----------+
|  4|   Shannon|O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|578|    Mignon|   Debnam|Female|Guajará Mirim|       Social Worker|$81607.74|-10.789321|-65.3301049|
|641|    Gannon|Heinreich|  Male|    Gualmatán|Analog Circuit De...|$88032.41|  0.919645| -77.567138|
+---+----------+---------+------+-------------+--------------------+---------+----------+-----------+



In [ ]:
df_filter = df2.filter(df2.first_name.startswith("Sho"))
df_filter.show()

+---+----------+----------+------+--------------+-------------------+---------+-----------+-----------+
| id|first_name| last_name|gender|          City|           JobTitle|   Salary|   Latitude|  Longitude|
+---+----------+----------+------+--------------+-------------------+---------+-----------+-----------+
|887| Shoshanna|Bransgrove|Female|Rio das Ostras|Software Consultant|$47099.85|-22.4206096|-41.8625084|
+---+----------+----------+------+--------------+-------------------+---------+-----------+-----------+



In [ ]:
df_filter = df2.filter(df2.id.between(20,30))
df_filter.show()

+---+----------+---------+------+-----------+--------------------+---------+----------+-----------+
| id|first_name|last_name|gender|       City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+---------+------+-----------+--------------------+---------+----------+-----------+
| 20|    Ettore| Gerriets|  Male|      Pedra| Staff Accountant IV|$73921.33|40.7172049| -8.3625148|
| 21|      Alon| Chasteau|  Male|      Xin’e|     Web Developer I|$62755.85|  49.16291|  127.98658|
| 22|   Guthrey|   Johnke|  Male|     Xijiao|         Developer I|$13708.03|40.7576842| 23.1342184|
| 23|   Lurleen|  Janczak|Female|  Rozkishne|    Web Developer IV|$29384.72|48.4902808| 39.2712224|
| 24|    Nichol|   Holtum|Female|      Město|           Paralegal|$27797.98|49.9718258| 12.8846277|
| 25|     Shaun|   Bridle|Female|      Xitou| Office Assistant IV|$71279.60| 29.866917| 118.403786|
| 26|   Leandra|   Anfrey|Female|    Isfahan|VP Product Manage...|$30201.32|32.6546275| 51.6679826|


In [ ]:
df_filter = df2.filter(df2.first_name.isin("Cherey","Nichol","Anitha"))
df_filter.show()

+---+----------+---------+------+--------+--------------------+---------+----------+-----------+
| id|first_name|last_name|gender|    City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+---------+------+--------+--------------------+---------+----------+-----------+
| 24|    Nichol|   Holtum|Female|   Město|           Paralegal|$27797.98|49.9718258| 12.8846277|
| 30|    Cherey|    Liger|Female|   Itami|     Design Engineer|$94566.69|34.7791206|135.4203895|
|967|    Cherey|  Spittal|Female|Kenscoff|Automation Specia...|$25525.55|18.4521371|-72.2867105|
+---+----------+---------+------+--------+--------------------+---------+----------+-----------+



In [ ]:
df_filter = df2.select("first_name",df2.first_name.substr(2,10).alias("sub name"))
df_filter.show()

+----------+--------+
|first_name|sub name|
+----------+--------+
|   Melinde|  elinde|
|  Kimberly| imberly|
|    Alvera|   lvera|
|   Shannon|  hannon|
|  Sherwood| herwood|
|     Maris|    aris|
|     Masha|    asha|
|   Goddart|  oddart|
|      Roth|     oth|
|      Bran|     ran|
|    Kylynn|   ylynn|
|       Rey|      ey|
|      Kerr|     err|
|    Mickie|   ickie|
|    Kaspar|   aspar|
|    Norbie|   orbie|
|    Claude|   laude|
|     Thain|    hain|
|  Tiffanie| iffanie|
|    Ettore|   ttore|
+----------+--------+
only showing top 20 rows



In [ ]:
df_filter = df2.filter(df2.last_name.like("Mac%") & df2.gender.like("F%"))
df_filter.show()

+---+----------+-----------+------+-----------------+--------------------+---------+-----------+-----------+
| id|first_name|  last_name|gender|             City|            JobTitle|   Salary|   Latitude|  Longitude|
+---+----------+-----------+------+-----------------+--------------------+---------+-----------+-----------+
| 56|   Goldina| MacFarlane|Female|            Gaohe|      Research Nurse|$25341.68|  36.145932|  113.01869|
|182|  Rochella|MacAllister|Female|          Sacanta|  VP Quality Control|$78262.15|-31.4325479|-64.1305924|
|280|     Carri|     Machel|Female|Reckange-sur-Mess|      Accountant III|$15589.56| 49.5636147|  6.0123531|
|775|   Alverta|   MacNulty|Female|       Megalópoli| Geological Engineer|$17299.62|  37.401244| 22.1364871|
|825|     Abbye| MacAndreis|Female|        Chandmanĭ|Accounting Assist...|$87792.30| 45.3369621| 97.9685657|
+---+----------+-----------+------+-----------------+--------------------+---------+-----------+-----------+



In [ ]:
df_filter = df2.filter((df2.id >= 40) & (df2.id <=50))
df_filter.show()

+---+----------+----------+------+------------+--------------------+---------+-----------+-----------+
| id|first_name| last_name|gender|        City|            JobTitle|   Salary|   Latitude|  Longitude|
+---+----------+----------+------+------------+--------------------+---------+-----------+-----------+
| 40| Pierrette|     Calow|Female|   Kaustinen|     Legal Assistant|$85120.77|  63.537266| 23.6993197|
| 41|   Deirdre|Codrington|Female| Buena Vista| Software Engineer I|$85861.33|-17.4586651|-63.6691522|
| 42|   Chelsie|   Yewdell|Female|     Klippan|         Engineer IV|$77039.47| 56.1357417| 13.1119454|
| 43|   Hadrian|    Crumpe|  Male|       Rosso|Physical Therapy ...|$36879.44| 16.5163413| -15.802612|
| 44|   Orville| Rigardeau|  Male|     Kokotów|     Data Coordiator|$20350.35| 50.0142619|  20.096687|
| 45| Gabrielle|  Tippings|Female|    Huangshi|              Editor|$98690.34|   30.20003| 115.038835|
| 46|    Sollie|    D'Acth|  Male|      Mozdok|     Design Engineer|$3138

** Running SQL on Dataframes**

In [ ]:
df2.registerTempTable("Original") #register the dataframe as a temporary table for running sql queries

In [ ]:
query1 = spark.sql("Select * from Original")
query1.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [ ]:
query2 = spark.sql("select concat(first_name,' ' ,last_name) as fullname from Original where gender='Female'")
query2.show()

+-------------------+
|           fullname|
+-------------------+
|  Melinde Shilburne|
|Kimberly Von Welden|
|    Alvera Di Boldi|
|         Maris Folk|
|       Masha Divers|
|     Kylynn Lockart|
|         Rey Meharg|
|      Claude Briant|
|  Tiffanie Pattison|
|    Lurleen Janczak|
|      Nichol Holtum|
|       Shaun Bridle|
|     Leandra Anfrey|
|    Jaquelyn Hazard|
|  Prudence Honacker|
|       Cherey Liger|
|          Neda Krop|
|    Barbi Fattorini|
|   Lonnie Townshend|
|    Valida Salzberg|
+-------------------+
only showing top 20 rows



In [ ]:
df5 = df5.withColumn("Monthly_salary",df5.clean_salary/12)
df5.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+--------------+----------+-----------------+----------+-----------+-----------+------------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|clean_salary_2|new_salary|     new_latitude|Fem_salary|male_salary|diff Salary|    Monthly_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+--------------+----------+-----------------+----------+-----------+-----------+------------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|      57438.18|  57438.18|       50.5774075|  57438.18|          0|   57438.18|          4786.515|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$

In [ ]:
df5 = df5.withColumn("is_female",when(df5.gender=='Female',"Female").otherwise("Not Female"))
df5.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+--------------+----------+-----------------+----------+-----------+-----------+------------------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|clean_salary_2|new_salary|     new_latitude|Fem_salary|male_salary|diff Salary|    Monthly_salary| is_female|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+--------------+----------+-----------------+----------+-----------+-----------+------------------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|      57438.18|  57438.18|       50.5774075|  57438.18|          0|   57438.18|          4786.515|    Female|
|  2|  Kimberly|Von Welden|F

In [ ]:
df_sum = df5.groupBy().sum("Monthly_salary")
df_sum.show()

+-------------------+
|sum(Monthly_salary)|
+-------------------+
| 4617107.3549999995|
+-------------------+



In [ ]:
df_sum = df5.groupBy("gender").sum("Monthly_salary")
df_sum.show()

+------+-------------------+
|gender|sum(Monthly_salary)|
+------+-------------------+
|Female| 2273487.7158333347|
|  Male| 2343619.6391666676|
+------+-------------------+



In [ ]:
df_sum = df5.groupBy("gender","city").agg(sum("Monthly_salary").alias("Total Monthly Salary"),
                                   avg("Monthly_salary").alias("Average Monthly Salary"),
                                   count("Monthly_salary").alias("Number of Employees"),
                                   min("Monthly_salary").alias("Minimum salary"),
                                   max("Monthly_salary").alias("Maximum Salary"))
df_sum.show()

+------+-----------------+--------------------+----------------------+-------------------+------------------+------------------+
|gender|             city|Total Monthly Salary|Average Monthly Salary|Number of Employees|    Minimum salary|    Maximum Salary|
+------+-----------------+--------------------+----------------------+-------------------+------------------+------------------+
|Female|      Trollhättan|   8885.280833333332|     4442.640416666666|                  2|         2235.8725| 6649.408333333333|
|  Male|          Wenshao|  1578.4591666666665|    1578.4591666666665|                  1|1578.4591666666665|1578.4591666666665|
|Female|            Lanas|  1147.1583333333333|    1147.1583333333333|                  1|1147.1583333333333|1147.1583333333333|
|  Male|            Mörön|   6495.006666666667|     6495.006666666667|                  1| 6495.006666666667| 6495.006666666667|
|Female|             Same|   6114.144166666666|     6114.144166666666|                  1| 6114.1

In [ ]:
df5.write.csv("output.csv")

In [ ]:
df5.write.json("output.json")
# df5.write.parquet("output.parquet")

AnalysisException: Attribute name "diff Salary" contains invalid character(s) among " ,;{}()\n\t=". Please use alias to rename it.

### Challenge

In [ ]:
ch_df = spark.read.csv("/content/challenge.csv", header=True)
ch_df.show()

+---------------+--------------+-----------------+----------+
|     ip_address|       Country|      Domain Name|Bytes_used|
+---------------+--------------+-----------------+----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|
| 119.239.207.13|         China|         youtu.be|        51|
|  68.69.217.210|         China|        adobe.com|        10|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|
|   211.13.10.68|     Indonesia|          hud.gov|        29|
|   239.80.21.97|      Suriname|       smh.com.au|       218|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|
| 127.242.24.138|         China| surveymonkey.com|       123|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|
|   237.54.11.63|         China|       amazon.com|        83|
| 252.141.157.25|         Japan|      cornell.edu|       374|
|185.220.128.248|       Belgium|       weebly.com|       389|
|   151.77.19.45|   Afghanistan|independent.co.uk|       282|
|  9.161

In [ ]:
ch_df.count()

1000

create a new column to mark nativity is Mexico or not

In [ ]:
ch_df = ch_df.withColumn("Mexico",when(ch_df.Country=="Mexico","yes").otherwise("No"))
ch_df.show()

+---------------+--------------+-----------------+----------+------+
|     ip_address|       Country|      Domain Name|Bytes_used|Mexico|
+---------------+--------------+-----------------+----------+------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|    No|
| 119.239.207.13|         China|         youtu.be|        51|    No|
|  68.69.217.210|         China|        adobe.com|        10|    No|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|    No|
|   211.13.10.68|     Indonesia|          hud.gov|        29|    No|
|   239.80.21.97|      Suriname|       smh.com.au|       218|    No|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|    No|
| 127.242.24.138|         China| surveymonkey.com|       123|    No|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|    No|
|   237.54.11.63|         China|       amazon.com|        83|    No|
| 252.141.157.25|         Japan|      cornell.edu|       374|    No|
|185.220.128.248|       Belgium|  

based on the nativity , find the total bytes used

In [ ]:
ch_df.groupBy("Mexico").agg(sum("Bytes_used").alias("Total Bytes Used")).show()


+------+----------------+
|Mexico|Total Bytes Used|
+------+----------------+
|    No|        508076.0|
|   yes|          6293.0|
+------+----------------+



no of unique ip address in each country sorted in order of no:

In [ ]:
ch_df.groupBy("Country").agg(countDistinct("ip_address").alias("No of Ip adds")).sort(col("No of Ip adds").desc()).show()

+--------------+-------------+
|       Country|No of Ip adds|
+--------------+-------------+
|         China|          172|
|     Indonesia|          114|
|   Philippines|           65|
|        Russia|           56|
|        Brazil|           35|
|        Poland|           31|
|        Sweden|           28|
|         Japan|           25|
|Czech Republic|           23|
|      Portugal|           23|
|        France|           21|
|          Peru|           19|
|      Colombia|           17|
| United States|           15|
|       Ukraine|           14|
|     Argentina|           14|
|        Mexico|           13|
|      Thailand|           12|
|        Canada|           11|
|       Nigeria|           11|
+--------------+-------------+
only showing top 20 rows



In [ ]:
data = [("Alice", 34, "2021-01-01"), ("Bob", 45, "2020-05-12"), ("Cathy", 29, "2019-07-19")]
columns = ["Name", "Age", "JoinDate"]

# Create DataFrame
df = spark.createDataFrame(data, columns)

# Use various functions
df.select(
    col("Name"),
    col("Age"),
    date_format(col("JoinDate"), "yyyy-MM-dd").alias("FormattedDate"),
    current_date().alias("CurrentDate"),
    datediff(current_date(), col("JoinDate")).alias("DaysSinceJoin"),
    when(col("Age") > 40, "Senior").otherwise("Junior").alias("Category")
).show()


+-----+---+-------------+-----------+-------------+--------+
| Name|Age|FormattedDate|CurrentDate|DaysSinceJoin|Category|
+-----+---+-------------+-----------+-------------+--------+
|Alice| 34|   2021-01-01| 2024-07-05|         1281|  Junior|
|  Bob| 45|   2020-05-12| 2024-07-05|         1515|  Senior|
|Cathy| 29|   2019-07-19| 2024-07-05|         1813|  Junior|
+-----+---+-------------+-----------+-------------+--------+



In [ ]:
df.select(
    col("Name"),
    col("JoinDate")).show()

+-----+----------+
| Name|  JoinDate|
+-----+----------+
|Alice|2021-01-01|
|  Bob|2020-05-12|
|Cathy|2019-07-19|
+-----+----------+

